In [11]:
import requests, csv, pandas as pd, pprint, time
from bs4 import BeautifulSoup
import lxml, html5lib
import re
from IPython.display import display, HTML

data_dict = {
    "name": [],
    "date": [],
    "platform": [],
    "score": [],
    "url": [],
    "ratings": [],
    "userscore": [],
}  # Data Structure


def webpage(
    pageNum, system
):  # function that navigates the metacritic SRP(Search Results Pages) based on the page number
    url = (
        "https://www.metacritic.com/browse/games/score/metascore/all/"
        + str(system)
        + "/filtered?view=condensed&page="
        + str(pageNum)
    )
    userAgent = {"User-agent": "Mozilla/5.0"}
    response = requests.get(url, headers=userAgent)
    return response


def numberPages(
    response,
):  # Helper Function that determines how many pages are in a SRP to know how many times to run scrapper function
    soup = BeautifulSoup(response.text, "html.parser")
    pages = soup.find_all("li", {"class": "page last_page"})
    try:
        pagesCleaned = pages[0].find("a", {"class": "page_num"}).text
    except:
        pagesCleaned = "1"
    return pagesCleaned


def scrapper(num_loops, content):
    tblnum = 0
    while tblnum < num_loops:
        # get Game name
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for a in td[1].find_all("a", {"class": "title"}):
                data_dict["name"].append(a.find("h3").text)

        # get Game release date
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for date in td[1].find_all("span", {"class": ""}):
                data_dict["date"].append(date.text)

        # get platform
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for platform in td[1].find_all("span", {"class": "data"}):
                data_dict["platform"].append(platform.text.strip())

        # get Game score
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for user in td[0].find_all("div", {"class": "metascore_w"}):
                data_dict["score"].append(user.text.strip())

        # getting game url
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for a in td[1].find_all("a", {"class": "title"}, href=True):
                data_dict["url"].append(a["href"])
                # print(a['href'])
                game_soup = BeautifulSoup(
                    requests.get(
                        "https://www.metacritic.com" + str(a["href"]) + "/user-reviews",
                        headers={"User-agent": "Mozilla/5.0"},
                    ).text,
                    "html.parser",
                )
                game_ratings = game_soup.find("strong", text=re.compile("Ratings"))
                if game_ratings:
                    game_ratings = (
                        game_ratings.get_text().strip().replace(" Ratings", "")
                    )
                else:
                    game_ratings = "0"
                # print(game_ratings)
                data_dict["ratings"].append(game_ratings)

        # get Game userscore
        table_rows = content[tblnum].find_all("tr")
        for tr in table_rows:
            td = tr.find_all("td")
            for score in td[1].find_all("div", {"class": "metascore_w"}):
                data_dict["userscore"].append(score.text)
        tblnum += 1


def pages(
    lastPageNum, system
):  # Function that returns the html(code) and initiates the web scrapper
    currentPage = 0
    while currentPage < int(lastPageNum):
        url = url = (
            "https://www.metacritic.com/browse/games/score/metascore/all/"
            + str(system)
            + "/filtered?view=condensed&page="
            + str(currentPage)
        )
        userAgent = {"User-agent": "Mozilla/5.0"}
        response = requests.get(url, headers=userAgent)
        soup = BeautifulSoup(response.text, "html.parser")
        content = soup.find_all("table")

        num_loops = len(content)
        # print(num_loops)
        scrapper(num_loops, content)
        # print(data_dict)
        currentPage += 1
        time.sleep(6)


def main():
    systems = ["switch"]
    for system in systems:
        # numPage = (numberPages(webpage(0,system)))
        numPage = 6 # cut down on runtime
        pages(int(numPage), system)
        time.sleep(5)
    xData = pd.DataFrame.from_dict(data_dict)
    xData.to_csv("mc_all_time.csv")


main()

video_games = pd.DataFrame.from_dict(data_dict)
video_games = video_games[~video_games["userscore"].isin(["tbd"])]
video_games = video_games[~video_games["score"].isin(["tbd"])]
video_games = video_games[~video_games["ratings"].isin(["tbd"])]
video_games = video_games[video_games["userscore"].astype(float) >= 7.5]
video_games = video_games[video_games["score"].astype(float) >= 75]
video_games = video_games[video_games["ratings"].astype(float) >= 150]

display(HTML(video_games.drop(['url'], axis=1).to_html()))


,name,date,platform,score,ratings,userscore
0,The Legend of Zelda: Breath of the Wild,"March 3, 2017",Switch,97,16753,8.7
1,Super Mario Odyssey,"October 27, 2017",Switch,97,5947,8.9
2,Divinity: Original Sin II - Definitive Edition,"September 4, 2019",Switch,93,301,8.5
3,Hades,"September 17, 2020",Switch,93,507,9.0
4,Ori and the Will of the Wisps,"September 17, 2020",Switch,93,264,8.8
5,Undertale,"September 18, 2018",Switch,93,395,8.1
6,Super Smash Bros. Ultimate,"December 7, 2018",Switch,93,3021,8.6
7,Celeste,"January 25, 2018",Switch,92,899,8.6
8,Bayonetta 2,"February 16, 2018",Switch,92,464,8.5
9,Mario Kart 8 Deluxe,"April 28, 2017",Switch,92,2096,8.6
